In [1]:
import requests
import numpy, pandas
import json
import random
from enum import Enum

In [59]:
class msg_builder:

    headers = {}
    msg_body = {}
    
    def __init__(self, token, message):
        self.headers = { "Authorization": "Bearer " + token }
        self.msg_body = { 'message': message }
        
    def print_content(self):
        print(self.headers)
        print(self.msg_body)

    
    def post_msg(self):
        requests.post("https://notify-api.line.me/api/notify",
    headers = self.headers, data = self.msg_body)
        

class data_parser:
    
    def json_converter(self, json_data):
        return json.loads(json_data)

    
class data_requester:
    url = ''
    
    def __init__(self, url):
       self.url = url 
    
    def get_url_data(self):
        resp = requests.get(self.url);
        return resp
    
class url_type(Enum):
    Bwibbu_all = '/exchangeReport/BWIBBU_ALL'
    Stock_day_avg_all = '/exchangeReport/STOCK_DAY_AVG_ALL'
    Stock_day_all = '/exchangeReport/STOCK_DAY_ALL'
    Fmsrfk_all = '/exchangeReport/FMSRFK_ALL'
    Fmnptk_all = '/exchangeReport/FMNPTK_ALL'
    Mi_index = '/exchangeReport/MI_INDEX'
    Qfiis_cat = '/fund/MI_QFIIS_cat'
    Qfiis_sort_20 = '/fund/MI_QFIIS_sort_20'
    Twt88u = '/exchangeReport/TWT88U'
    Bfzfzu_t = '/Announcement/BFZFZU_T'
    Twtb4u = '/exchangeReport/TWTB4U'
    
class strategy:
    base_url = 'http://openapi.twse.com.tw/v1'
    
    def find_target_daily_volume_over_month_avg(self):
        output_dict = []
        
        daily_info_url = self.base_url + url_type.Stock_day_all.value
        monthly_info_url = self.base_url + url_type.Fmsrfk_all.value
        
        daily_info_data = data_parser().json_converter((data_requester(daily_info_url).get_url_data()).text)
        monthly_info_data = data_parser().json_converter((data_requester(monthly_info_url).get_url_data()).text)
        
        data_size = len(daily_info_data)
        
        for i in range(data_size):
            if (daily_info_data[i]['TradeVolume'] == ''): continue
            daily_trade_volume = float(daily_info_data[i]['TradeVolume'])
            monthly_trade_volume = float(monthly_info_data[i]['TradeVolumeB'])
            
            daily_avg_tranc_vol = (daily_trade_volume/float(daily_info_data[i]['Transaction']))
            monthly_avg_tranc_vol = (monthly_trade_volume/float(monthly_info_data[i]['Transaction']))
                                   
            if (daily_trade_volume > monthly_trade_volume) and (daily_avg_tranc_vol > monthly_avg_tranc_vol):
                output_dict.append(daily_info_data[i])
        
        return output_dict
    
    def find_target_daily_closin_price_over_month_avg(self):
        output_dict = []
        
        daily_info_url = self.base_url + url_type.Stock_day_all.value
        monthly_info_url = self.base_url + url_type.Fmsrfk_all.value
        
        daily_info_data = data_parser().json_converter((data_requester(daily_info_url).get_url_data()).text)
        monthly_info_data = data_parser().json_converter((data_requester(monthly_info_url).get_url_data()).text)
        
        data_size = len(daily_info_data)
        
        for i in range(data_size):
            if (daily_info_data[i]['TradeVolume'] == ''): continue
                
            dialy_closin_price = float(daily_info_data[i]['ClosingPrice'])
            monthly_closin_price = float(monthly_info_data[i]['WeightedAvgPriceAB'])
            
            if(dialy_closin_price > monthly_closin_price):
                output_dict.append(daily_info_data[i])
        return output_dict

In [60]:
data = strategy().find_target_daily_closin_price_over_month_avg()

In [62]:
data

[{'Code': '0050',
  'Name': '元大台灣50',
  'TradeVolume': '6394176',
  'TradeValue': '733944261',
  'OpeningPrice': '114.00',
  'HighestPrice': '115.25',
  'LowestPrice': '113.95',
  'ClosingPrice': '115.00',
  'Change': '1.7500',
  'Transaction': '6619'},
 {'Code': '0051',
  'Name': '元大中型100',
  'TradeVolume': '12886',
  'TradeValue': '678791',
  'OpeningPrice': '52.55',
  'HighestPrice': '52.85',
  'LowestPrice': '52.55',
  'ClosingPrice': '52.55',
  'Change': '0.0500',
  'Transaction': '154'},
 {'Code': '0052',
  'Name': '富邦科技',
  'TradeVolume': '179268',
  'TradeValue': '17584067',
  'OpeningPrice': '97.75',
  'HighestPrice': '98.55',
  'LowestPrice': '97.30',
  'ClosingPrice': '98.30',
  'Change': '1.6000',
  'Transaction': '282'},
 {'Code': '0053',
  'Name': '元大電子',
  'TradeVolume': '5249',
  'TradeValue': '288119',
  'OpeningPrice': '55.25',
  'HighestPrice': '55.25',
  'LowestPrice': '55.05',
  'ClosingPrice': '55.05',
  'Change': '0.7500',
  'Transaction': '1019'},
 {'Code': '005

In [33]:
url = 'http://openapi.twse.com.tw/v1/exchangeReport/STOCK_DAY_AVG_ALL'

In [44]:
resp = data_requester(url).get_url_data()
month_data = data_parser().json_converter(resp.text)


In [45]:
month_data

[{'Code': '0050',
  'Name': '元大台灣50',
  'ClosingPrice': '115.30',
  'MonthlyAveragePrice': '117.10'},
 {'Code': '0051',
  'Name': '元大中型100',
  'ClosingPrice': '53.00',
  'MonthlyAveragePrice': '53.33'},
 {'Code': '0052',
  'Name': '富邦科技',
  'ClosingPrice': '98.55',
  'MonthlyAveragePrice': '99.99'},
 {'Code': '0053',
  'Name': '元大電子',
  'ClosingPrice': '55.30',
  'MonthlyAveragePrice': '56.10'},
 {'Code': '0055',
  'Name': '元大MSCI金融',
  'ClosingPrice': '21.81',
  'MonthlyAveragePrice': '22.02'},
 {'Code': '0056',
  'Name': '元大高股息',
  'ClosingPrice': '25.61',
  'MonthlyAveragePrice': '25.76'},
 {'Code': '0057',
  'Name': '富邦摩台',
  'ClosingPrice': '82.55',
  'MonthlyAveragePrice': '83.40'},
 {'Code': '0061',
  'Name': '元大寶滬深',
  'ClosingPrice': '18.65',
  'MonthlyAveragePrice': '18.36'},
 {'Code': '006203',
  'Name': '元大MSCI台灣',
  'ClosingPrice': '56.25',
  'MonthlyAveragePrice': '56.66'},
 {'Code': '006204',
  'Name': '永豐臺灣加權',
  'ClosingPrice': '73.85',
  'MonthlyAveragePrice': '74.70'